# Assignment-4 (Perceptron, SVM, and Neural Networks)

This part of the assignment shall require you to code the perceptron classifier from the ground up. The perceptron classifier takes uses the weighted sum of input features, uses a threshold to classify between two classes. Usually these classes are -1 and 1 while the threshold is 0

In [12]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.svm import SVC

%matplotlib inline

# 1. Prepare the dataset

In [13]:
# import the assigned dataset
columns = ["fLength","fWidth","fSize","fConc","fConc1","fAsym","fM3Long","fM3Trans","fAlpha","fDist","targets"]
print(len(columns))

df = pd.read_csv(
    filepath_or_buffer='https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data', 
    header=None, 
    sep=',')

11


In [14]:
# Preprocess the dataset with normalization and label encoding
df.columns = columns
df.dropna(inplace=True,how="all")
df = df[["fLength","fWidth","fSize","fConc","fConc1","fAsym","fM3Long","fM3Trans","targets"]]
df["targets"]= df["targets"].astype('category').cat.codes
df_0 = df[df['targets'].apply(lambda x:x==0)].head(500)
df_1 = df[df['targets'].apply(lambda x:x==1)].head(500)
df_final = [df_0,df_1]
pd.concat(df_final)

# df= df[:350]+df[18950:]
x = df
y = df["targets"]
x.drop("targets",axis=1,inplace=True)
x =x.values
y = y.values
y=y.reshape(len(y),1)
X_norm = StandardScaler().fit_transform(x)

dataset = np.hstack((X_norm, y))
dataset = dataset.tolist()

<ipython-input-14-b8b2152667e3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["targets"]= df["targets"].astype('category').cat.codes
/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [15]:
# Make a train-test split with 80% to training and 20% to testing.
x_full_train,x_full_test,y_full_train,y_full_test = train_test_split(X_norm,y, test_size=0.01, random_state=None, shuffle=True,stratify=y)


y_full_train = y_full_train.reshape(len(y_full_train),)
y_full_test = y_full_test.reshape(len(y_full_test),)


In [ ]:
# Normalize numerical features and encode the categorical features (if any)


As this is a classification problem, the target variables must be categorical. Using the feature and target variable information, preprocess the dataset to use given features.

This step might require students to scale the features, one-hot encode categorical FEATURES (if any).



For feature scaling and one-hot encoding, go through:

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

# 2: Creating a perceptron model for the processed dataset

The perceptron.py file in the resources provides functions to code the perceptron model from scratch.

Using the file as reference, write the functions:

1. cross_validation_split
2. accuracy_metric
3. evaluate_algorithm
4. predict
5. train_weights
6. perceptron

This step is aimed at providing a comprehensive understanding of the internal functioning of a perceptron model.

In [16]:
# your code for step 2
from random import seed
from random import randrange
from csv import reader


def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split


def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0


def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

def predict(row, weights):
	activation = weights[0]
	for i in range(len(row)-1):
		activation += weights[i + 1] * row[i]
	return 1.0 if activation >= 0.0 else 0.0

def train_weights(train, l_rate, n_epoch):
	weights = [0.0 for i in range(len(train[0]))]
	for epoch in range(n_epoch):
		for row in train:
			prediction = predict(row, weights)
			error = row[-1] - prediction
			weights[0] = weights[0] + l_rate * error
			for i in range(len(row)-1):
				weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
	return weights

def perceptron(train, test, l_rate, n_epoch):
	predictions = list()
	weights = train_weights(train, l_rate, n_epoch)
	for row in test:
		prediction = predict(row, weights)
		predictions.append(prediction)
	return(predictions)

In [17]:
# your code for step 2
def perceptron_call(n_fold):
  l_rate = 0.01
  n_epoch = 100
  scores = evaluate_algorithm(dataset, perceptron, n_fold, l_rate, n_epoch)
  print('Scores: %s' % scores)
  print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

perceptron_call(3) 

Scores: [63.23343848580441, 47.85488958990536, 67.36593059936908]
Mean Accuracy: 59.485%


# (Bonus) Perceptron model with relaxation

Using Relaxation (the descent theorem), compare the performance of perceptron model with and without relaxation (refer class lectures, slides for details on relaxation).

Make modifications to the loss function in the perceptron model.

In [18]:
# code for relaxation for the perceptron model

def trainRelaxation(train, l_rate, n_epoch):
	weights = [0.0 for i in range(len(train[0]))]
	for epoch in range(n_epoch):
		for row in train:
			prediction = predict(row, weights)
			error = row[-1] - prediction
			activation = weights[0]
			for i in range(len(row)-1):
				activation += weights[i + 1] * row[i]
			weights[0] = weights[0] + l_rate * 2 * activation * error
			for i in range(len(row)-1):
				weights[i + 1] = weights[i + 1] + l_rate * 2 * activation  * error * row[i]
	return weights

def perceptronRelaxation(train, test, l_rate, n_epoch):
	predictions = list()
	weights = trainRelaxation(train, l_rate, n_epoch)
	for row in test:
		prediction = predict(row, weights)
		predictions.append(prediction)
	return(predictions)  

In [19]:
n_fold = 3
l_rate = 0.01
n_epoch = 100
scores = evaluate_algorithm(dataset, perceptronRelaxation, n_fold, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [35.61514195583596, 34.81072555205048, 35.063091482649845]
Mean Accuracy: 35.163%


# 3: Batch size for perceptron model

Experiment with different batch sizes in the perceptron model (eg: 1, 4, 8).

Report (with figures) the difference in performance when using different batch sizes. Inferences without plots might not be awarded points.

Report the accuracies for various combinations of batch sizes.

In [36]:
# code for step 3
perceptron_call(2)

Scores: [60.56782334384858, 69.94742376445846]
Mean Accuracy: 65.258%


In [37]:
# code for step 3
perceptron_call(4)

Scores: [52.008412197686646, 49.50578338590957, 53.14405888538381, 60.1472134595163]
Mean Accuracy: 53.701%


In [38]:
# code for step 3
perceptron_call(5)

Scores: [62.56572029442692, 66.19348054679286, 66.40378548895899, 66.92954784437434, 51.34069400630915]
Mean Accuracy: 62.687%


# 4. SVM's

### **Note : You are allowed to use sklearn's SVC classifier for steps 4.1 through 4.3**

https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

---

# 4.1 Linear SVM


---


# Step 1: Implement a linear SVM model to classify the data points. (Look into the 'kernel' parameter).

In [26]:
# linear SVM
svm_lin_clf = SVC(kernel="linear", C=1.0)

# Step 2: Train the model

In [27]:
# training - linear SVM
svm_lin_clf.fit(x_full_train, y_full_train)

SVC(kernel='linear')

# Step 3: Predict for the test points using the model trained in the previous step

In [29]:
# predict - linear SVM
svm_lin_acc = svm_lin_clf.score(x_full_test, y_full_test) * 100
svm_lin_predict = svm_lin_clf.predict(x_full_test)
print(svm_lin_predict)

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1
 1 0 0 0 0 0]


# 4.2 Kernel SVM - Polynomial kernel

---



# Step 1: Implement a kernel SVM model with a polynomial kernel to classify the data points.

In [30]:
# kernel SVM - polynomial kernel
svm_poly_clf = SVC(kernel="poly", coef0=1, C=3)

# Step 2: Train the model

In [32]:
# # training - kernel SVM
svm_poly_clf.fit(x_full_train, y_full_train)

SVC(C=3, coef0=1, kernel='poly')

numpy.ndarray

# Step 3: Predict for the test points using the model trained in the previous step

In [33]:
# predict - kernel SVM
svm_poly_acc = svm_poly_clf.score(x_full_test, y_full_test) * 100
svm_poly_predict = svm_poly_clf.predict(x_full_test)
print(svm_poly_predict)

[0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0
 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0
 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 1
 1 0 0 0 0 0]


# 4.3 Kernel SVM - Gaussian kernel

---



# Step 1: Implement a kernel SVM model with a gaussian (Radian Basis function) kernel to classify the data points.

In [36]:
# kernel SVM - gaussian kernel
svm_rbf_clf = SVC(kernel="rbf", C=1.5)

# Step 2: Train the model

In [37]:
# training - kernel SVM
svm_rbf_clf.fit(x_full_train, y_full_train)

SVC(C=1.5)

# Step 3: Predict for the test points using the model trained in the previous step

In [38]:
# predict - kernel SVM
svm_rbf_acc = svm_rbf_clf.score(x_full_test, y_full_test) * 100
svm_rbf_predict = svm_rbf_clf.predict(x_full_test)
print(svm_rbf_predict)

[0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1
 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0
 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0
 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 1
 1 0 0 0 0 0]


# 4.4 - Evaluation


---


# Take the results from each predict step under sections 4.1, 4.2 and 4.3. Consider accuracy as the evaulation metric. Print the accuracies for each of the 3 SVM models.

Note: Do not use functions from sklearn.metrics

In [39]:
print("The accuracies of the above classifiers are as follows: ")
print("Accuracy for linear kernel: ", svm_lin_acc)
print("Accuracy for polynomial kernel: ", svm_poly_acc)
print("Accuracy for Gaussian kernel: ", svm_rbf_acc)

The accuracies of the above classifiers are as follows: 
Accuracy for linear kernel:  75.39267015706807
Accuracy for polynomial kernel:  78.53403141361257
Accuracy for Gaussian kernel:  78.53403141361257


In [ ]:
# space for any imports for the following steps

# 5. Neural Networks



---



# 5.1 Single layer neural network

You can use either PyTorch or Tensorflow for the implementation

---

# Step 1: Implement a single layer neural network to classify the data points

In [40]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


def logit(z):
    return 1 / (1 + np.exp(-z))

def relu(z):
    return np.maximum(0, z)

def derivative(f, z, eps=0.000001):
    return (f(z + eps) - f(z - eps))/(2 * eps)
def heaviside(z):
    return (z >= 0).astype(z.dtype)

def sigmoid(z):
    return 1/(1+np.exp(-z))

def mlp_xor(x1, x2, activation=heaviside):
    return activation(-activation(x1 + x2 - 1.5) + activation(x1 + x2 - 0.5) - 0.5)

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

n_inputs = 8 
n_hidden1 = 300
n_outputs = 2

X = tf.compat.v1.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.compat.v1.placeholder(tf.int32, shape=(None), name="y")

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="kernel")
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

with tf.name_scope("dnn1"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden1, n_outputs, name="outputs")

with tf.name_scope("loss1"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

learning_rate = 0.01

with tf.name_scope("train1"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval1"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()


n_epochs = 40
batch_size = 50

def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch




Instructions for updating:
non-resource variables are not supported in the long term


# Step 2: Train the model

In [41]:
with tf.Session() as sess1:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(x_full_train, y_full_train, batch_size):
            sess1.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        print(epoch, "Batch accuracy:", acc_batch)

    save_path = saver.save(sess1, "./my_model1.ckpt")

0 Batch accuracy: 0.8
1 Batch accuracy: 0.82
2 Batch accuracy: 0.86
3 Batch accuracy: 0.84
4 Batch accuracy: 0.78
5 Batch accuracy: 0.82
6 Batch accuracy: 0.9
7 Batch accuracy: 0.8
8 Batch accuracy: 0.76
9 Batch accuracy: 0.66
10 Batch accuracy: 0.84
11 Batch accuracy: 0.84
12 Batch accuracy: 0.74
13 Batch accuracy: 0.78
14 Batch accuracy: 0.78
15 Batch accuracy: 0.68
16 Batch accuracy: 0.84
17 Batch accuracy: 0.88
18 Batch accuracy: 0.9
19 Batch accuracy: 0.82
20 Batch accuracy: 0.84
21 Batch accuracy: 0.9
22 Batch accuracy: 0.84
23 Batch accuracy: 0.72
24 Batch accuracy: 0.88
25 Batch accuracy: 0.84
26 Batch accuracy: 0.86
27 Batch accuracy: 0.86
28 Batch accuracy: 0.86
29 Batch accuracy: 0.8
30 Batch accuracy: 0.78
31 Batch accuracy: 0.9
32 Batch accuracy: 0.88
33 Batch accuracy: 0.86
34 Batch accuracy: 0.82
35 Batch accuracy: 0.8
36 Batch accuracy: 0.76
37 Batch accuracy: 0.78
38 Batch accuracy: 0.86
39 Batch accuracy: 0.64


# Step 3: Predict for the test points using the model trained in the previous step

In [42]:
with tf.Session() as sess1:
    saver.restore(sess1, "./my_model1.ckpt") # or better, use save_path
    Z = logits.eval(feed_dict={X: x_full_test})
    y_pred = np.argmax(Z, axis=1)

print("Predicted classes:", y_pred[:20])
print("Actual classes:   ", y_full_test[:20])  

correct= 0
for i in range(len(y_pred)):
  if y_pred[i] == y_full_test[i]:
        correct += 1
accuracy1 = correct/len(y_pred)
accuracy1

Predicted classes: [1 0 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0]
Actual classes:    [0 0 1 1 0 0 0 0 1 1 1 0 0 0 0 0 1 1 1 1]


0.7486910994764397

# 5.2 Multi - Layer neural network

---

# Step 1: Implement a multi - layer neural network to classify the data points

Additional note: use methods to avoid overfitting appropriately

In [43]:
n_inputs = 8  
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 2


X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn2"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss2"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

learning_rate = 0.01

with tf.name_scope("train2"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval2"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()








# Step 2: Train the model

In [44]:
with tf.Session() as sess2:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(x_full_train, y_full_train, batch_size):
            sess2.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        print(epoch, "Batch accuracy:", acc_batch)

    save_path = saver.save(sess2, "./my_model2.ckpt")

0 Batch accuracy: 0.78
1 Batch accuracy: 0.74
2 Batch accuracy: 0.8
3 Batch accuracy: 0.78
4 Batch accuracy: 0.76
5 Batch accuracy: 0.88
6 Batch accuracy: 0.76
7 Batch accuracy: 0.9
8 Batch accuracy: 0.74
9 Batch accuracy: 0.74
10 Batch accuracy: 0.76
11 Batch accuracy: 0.78
12 Batch accuracy: 0.88
13 Batch accuracy: 0.8
14 Batch accuracy: 0.74
15 Batch accuracy: 0.72
16 Batch accuracy: 0.92
18 Batch accuracy: 0.84
19 Batch accuracy: 0.7
20 Batch accuracy: 0.76
21 Batch accuracy: 0.8
22 Batch accuracy: 0.86
23 Batch accuracy: 0.86
24 Batch accuracy: 0.76
25 Batch accuracy: 0.86
26 Batch accuracy: 0.76
27 Batch accuracy: 0.84
28 Batch accuracy: 0.84
29 Batch accuracy: 0.86
30 Batch accuracy: 0.7
31 Batch accuracy: 0.84
32 Batch accuracy: 0.74
33 Batch accuracy: 0.82
34 Batch accuracy: 0.88
35 Batch accuracy: 0.76
36 Batch accuracy: 0.86
37 Batch accuracy: 0.78
38 Batch accuracy: 0.8
39 Batch accuracy: 0.84


# Step 3: Predict for the test points using the model trained in the previous step

In [45]:
with tf.Session() as sess2:
    saver.restore(sess2, "./my_model2.ckpt") # or better, use save_path
    Z = logits.eval(feed_dict={X: x_full_test})
    y_pred = np.argmax(Z, axis=1)

print("Predicted classes:", y_pred[:20])
print("Actual classes:   ", y_full_test[:20])  

correct= 0
for i in range(len(y_pred)):
  if y_pred[i] == y_full_test[i]:
        correct += 1
accuracy2 = correct/len(y_pred)
accuracy2

Predicted classes: [0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0]
Actual classes:    [0 0 1 1 0 0 0 0 1 1 1 0 0 0 0 0 1 1 1 1]


0.7958115183246073

# 5.3 Multi - Layer neural network

---

# Step 1: Implement a multi - layer neural network to classify the data points

**Note :** This must have a different network architecture from the model under section 5.2

**Additional note:** use methods to avoid overfitting appropriately

In [46]:
n_inputs = 8 
n_hidden1 = 300
n_hidden2 = 100
n_hidden3 = 200
n_outputs = 2

X = tf.compat.v1.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.compat.v1.placeholder(tf.int32, shape=(None), name="y")


with tf.name_scope("dnn3"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1",
                           activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2",
                           activation=tf.nn.sigmoid)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3",
                           activation=tf.nn.relu)
    logits = neuron_layer(hidden3, n_outputs, name="outputs")

with tf.name_scope("loss3"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train3"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval3"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()    




# Step 2: Train the model

In [47]:
with tf.Session() as sess3:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(x_full_train, y_full_train, batch_size):
            sess3.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        print(epoch, "Batch accuracy:", acc_batch)

    save_path = saver.save(sess3, "./my_model3.ckpt")

0 Batch accuracy: 0.72
1 Batch accuracy: 0.8
2 Batch accuracy: 0.78
3 Batch accuracy: 0.9
4 Batch accuracy: 0.7
5 Batch accuracy: 0.74
6 Batch accuracy: 0.84
7 Batch accuracy: 0.78
8 Batch accuracy: 0.82
9 Batch accuracy: 0.82
10 Batch accuracy: 0.82
11 Batch accuracy: 0.78
12 Batch accuracy: 0.84
13 Batch accuracy: 0.88
14 Batch accuracy: 0.8
15 Batch accuracy: 0.86
16 Batch accuracy: 0.76
17 Batch accuracy: 0.82
18 Batch accuracy: 0.86
19 Batch accuracy: 0.82
20 Batch accuracy: 0.8
21 Batch accuracy: 0.84
22 Batch accuracy: 0.84
23 Batch accuracy: 0.84
24 Batch accuracy: 0.82
25 Batch accuracy: 0.8
26 Batch accuracy: 0.88
27 Batch accuracy: 0.82
28 Batch accuracy: 0.78
29 Batch accuracy: 0.76
30 Batch accuracy: 0.86
31 Batch accuracy: 0.88
32 Batch accuracy: 0.86
33 Batch accuracy: 0.82
34 Batch accuracy: 0.88
35 Batch accuracy: 0.74
36 Batch accuracy: 0.86
37 Batch accuracy: 0.76
38 Batch accuracy: 0.82
39 Batch accuracy: 0.8


# Step 3: Predict for the test points using the model trained in the previous step

In [48]:
with tf.Session() as sess3:
    saver.restore(sess3, "./my_model3.ckpt") # or better, use save_path
    Z = logits.eval(feed_dict={X: x_full_test})
    y_pred = np.argmax(Z, axis=1)

print("Predicted classes:", y_pred[:20])
print("Actual classes:   ", y_full_test[:20]) 

correct= 0
for i in range(len(y_pred)):
  if y_pred[i] == y_full_test[i]:
        correct += 1
accuracy3 = correct/len(y_pred)
accuracy3


Predicted classes: [0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0]
Actual classes:    [0 0 1 1 0 0 0 0 1 1 1 0 0 0 0 0 1 1 1 1]


0.7905759162303665

# 5.4 - Evaluation


---


# Take the results from each predict step under sections 5.1, 5.2 and 5.3. Consider accuracy as the evaulation metric. Print the accuracies for each of the 3 neural network architectures.

Note: Do not use functions from sklearn.metrics

In [50]:
print("The accuracy with single layer neural network is",accuracy1);
print("The accuracy from two layer neural network is",accuracy2);
print("The accuracy from three layer neural network is is",accuracy3);



The accuracy with single layer neural network is 0.7486910994764397
The accuracy from two layer neural network is 0.7958115183246073
The accuracy from three layer neural network is is 0.7905759162303665


# 6 - Paragraph questions

---



# Q1 : Briefly explain the methods you used to prevent overfitting for the models under section 5.2 and 5.3

***space for Q1's answer***

# Q2: Compare the performances of models under sections 2, 3, 4 and 5 namely perceptron, descent procedure, SVM's and neural networks. List down few points on why you think certain models performed better than others



***space for Q2's answer***